In [1]:
import copy

In [2]:
#the start and end time in a day
start_time = 360
end_time = 1320

In [3]:
#the dict of numbers of gates at each airport
gate_no = {'AUS':1, 'DAL':2, 'HOU':3}

#the dict of minimum ground time at each airport
ground_time = {'AUS':25, 'DAL':30, 'HOU':35}

#the dict of flight time among airports
flight_time = {('AUS','HOU'):45, ('HOU','AUS'):45, ('AUS','DAL'):50, ('DAL','AUS'):50, ('DAL','HOU'):65, ('HOU','DAL'):65, ('AUS','AUS'):0, ('DAL','DAL'):0, ('HOU','HOU'):0}

In [4]:
class flight:
    def __init__(self,tailNo=None, depPort=None, arrPort=None, depTime=None, arrTime=None, status=0):
        self.tailNo = tailNo
        self.depPort = depPort
        self.arrPort = arrPort
        self.depTime = depTime
        self.arrTime = arrTime
        self.status = status #flying status for the day, 0-flying, 1-stoped
    
    def __repr__(self):
        return repr((self.tailNo, self.depPort, self.arrPort, self.depTime, self.arrTime, self.status))

In [5]:
#initial flight schedule
T1 = flight('T1','AUS','HOU',360,405)
T2 = flight('T2','DAL','HOU',360,425)
T3 = flight('T3','DAL','HOU',360,425)
T4 = flight('T4','HOU','AUS',360,405)
T5 = flight('T5','HOU','DAL',360,425)
T6 = flight('T6','HOU','DAL',360,425)

#the list of all aircrafts
flights = [T1,T2,T3,T4,T5,T6]

#the list of flight schedule
flight_schedule = copy.deepcopy(flights)

#the lists include the arriving flights for airports. FIFO based on possible earlist depTime
portAUS = [T4]
portDAL = [T5,T6]
portHOU = [T1,T2,T3]

In [6]:
#from six aircrafts, find the minimum depTime after arriving 
def get_min_next_depTime(flights=[flight()]):
    min_next_depTime = end_time
    for flight in flights:
        if flight.arrTime + ground_time[flight.arrPort] < min_next_depTime and flight.status == 0:
            min_next_depTime = flight.arrTime + ground_time[flight.arrPort]
    return min_next_depTime

In [7]:
#based on flying status of six aircrafts, find today's falying stauts
def get_flying_status(flights=[flight()]):
    flying = 1
    for flight_status in flights:
        if flight_status.status <= flying:
            flying = flight_status.status
    return flying

In [8]:
#find if a port available and the minimum available time
def is_port_available(arrTime, airport, gates=[]):

    if len(gates) < gate_no[airport]:
        is_available = True
        min_avlTime = arrTime -1
        
    else:
        #minimum available time
        min_avlTime = end_time
        #possible arrive time if a flight leaves the gate
        pos_arrTime = end_time
        
        #find the min avlTime at an airport
        for flight in gates:
            if flight.status == 0:
                avlTime = flight.arrTime + ground_time[airport]
            else:
                avlTime = end_time
                
            if avlTime < min_avlTime:
                min_avlTime = avlTime
                #to be conservative, use the biggest flight time in estimation
                pos_arrTime = min_avlTime + 65
        
        #the airport will be available:
        #if the aircraft could arrive after the min_avlTime and before end_time
        #and a current aircraft could arrive at a new airport before end_time if it will departure
        if arrTime > min_avlTime and arrTime < end_time and pos_arrTime < end_time:
            is_available = True
        else:
            is_available = False

        #use end_time as min_avlTime for two scenarios 
        if arrTime > end_time or pos_arrTime > end_time:
            min_avlTime = end_time
    
    return (is_available, min_avlTime)

In [9]:
#find a target airport giving depPort and depTime
def get_arrPort(depPort, depTime):
        
    #dict will return 0 if depPort == arrPort
    arrTime_AUS = depTime + flight_time[(depPort,'AUS')]    
    arrTime_DAL = depTime + flight_time[(depPort,'DAL')] 
    arrTime_HOU = depTime + flight_time[(depPort,'HOU')]
                
    arrPort = ''
    new_depTime = depTime
    
    if depPort == 'HOU':
        isAvl_AUS, avlTime_AUS = is_port_available(arrTime_AUS, 'AUS', portAUS)
        isAvl_DAL, avlTime_DAL = is_port_available(arrTime_DAL, 'DAL', portDAL) 
        
        #when both ports are available
        if isAvl_AUS and isAvl_DAL:
            #AUS is preferred since the flight time is shorter
            if avlTime_AUS <= avlTime_DAL:
                arrPort = 'AUS'
            else:
                arrPort = 'DAL'
        #choose the one which is available
        elif isAvl_AUS and not isAvl_DAL:
            arrPort = 'AUS'
        elif isAvl_DAL and not isAvl_AUS:
            arrPort = 'DAL'
        #otherwise, choose the one that the avlTime is the smallest
        else: 
            #AUS is preferred since the flight time is shorter
            if avlTime_AUS <= avlTime_DAL and avlTime_AUS != end_time:
                arrPort = 'AUS'
                new_depTime = avlTime_AUS - flight_time[(depPort,'AUS')] + 1
            elif avlTime_DAL < avlTime_AUS and valTime_DAL != end_time:
                arrPort = 'DAL'
                new_depTime = avlTime_DAL - flight_time[(depPort,'DAL')] + 1
            #both avlTime == end_time, no flight should be arranged
            else:
                arrPort = 'HOU'
                new_depTime = end_time
                
    elif depPort == 'DAL':
        isAvl_AUS, avlTime_AUS = is_port_available(arrTime_AUS, 'AUS', portAUS)
        isAvl_HOU, avlTime_HOU = is_port_available(arrTime_HOU, 'HOU', portHOU) 

        #when both ports are available        
        if isAvl_AUS and isAvl_HOU:
            #AUS is preferred since the flight time is shorter
            if avlTime_AUS <= avlTime_HOU:
                arrPort = 'AUS'
            else:
                arrPort = 'HOU'
        #choose the one which is available                
        elif isAvl_AUS and not isAvl_HOU:
            arrPort = 'AUS'
        elif isAvl_HOU and not isAvl_AUS:
            arrPort = 'HOU'
        #otherwise, choose the one that the avlTime is the smallest            
        else: 
            #AUS is preferred since the flight time is shorter
            if avlTime_AUS <= avlTime_HOU and avlTime_AUS != end_time:
                arrPort = 'AUS'
                new_depTime = avlTime_AUS - flight_time[(depPort,'AUS')] + 1
            elif avlTime_HOU < avlTime_AUS and avlTime_HOU != end_time:
                arrPort = 'HOU'
                new_depTime = avlTime_HOU - flight_time[(depPort,'HOU')] + 1
            #both avlTime == end_time, no flight should be arranged                
            else:
                arrPort = 'DAL'
                new_depTime = end_time
                
    else:
        isAvl_DAL, avlTime_DAL = is_port_available(arrTime_DAL, 'DAL', portDAL)
        isAvl_HOU, avlTime_HOU = is_port_available(arrTime_HOU, 'HOU', portHOU) 
        
        #when both ports are available                
        if isAvl_DAL and isAvl_HOU:
            #HOU is preferred since the flight time is shorter
            if avlTime_HOU <= avlTime_DAL:
                arrPort = 'HOU'
            else:
                arrPort = 'DAL'
        #choose the one which is available                
        elif isAvl_DAL and not isAvl_HOU:
            arrPort = 'DAL'
        elif isAvl_HOU and not isAvl_DAL:
            arrPort = 'HOU'
        else: 
            #HOU is preferred since the flight time is shorter
            if avlTime_HOU <= avlTime_DAL and avlTime_HOU != end_time:
                arrPort = 'HOU'
                new_depTime = avlTime_HOU - flight_time[(depPort,'HOU')] + 1
            elif avlTime_DAL < avlTime_HOU and avlTime_DAL != end_time:
                arrPort = 'DAL'
                new_depTime = avlTime_DAL - flight_time[(depPort,'DAL')] + 1
            #both avlTime == end_time, no flight should be arranged                                
            else:
                arrPort = 'AUS'
                new_depTime = end_time
    
    return (arrPort, new_depTime)

In [10]:
#remove aircraft from the list of its departuring airport
def dep_airport(depPort, flight=flight()):
    if depPort == 'HOU':
        if flight in portHOU:
            portHOU.remove(flight)        
    elif depPort == 'DAL':
        if flight in portDAL:
            portDAL.remove(flight)
    else: 
        if flight in portAUS:
            portAUS.remove(flight)           

In [11]:
#update the list for arrving airport
def arr_airport(arrPort, flight=flight()):
    if arrPort == 'HOU':
        #there is empty gate
        if len(portHOU) < gate_no['HOU']:
            portHOU.append(flight)
        else:
            #from cfats which has not stoped flying, find the earlist departuring time            
            min_depTime = get_min_next_depTime(portHOU)
            #remove the flight which will depature firstly
            for arrFlight in portHOU:
                if arrFlight.arrTime + ground_time[arrFlight.arrPort] == min_depTime:
                    portHOU.remove(arrFlight)
            portHOU.append(flight)
                    
    elif arrPort == 'DAL':
        #there is empty gate
        if len(portDAL) < gate_no['DAL']:
            portDAL.append(flight)
        else:
            #from cfats which has not stoped flying, find the earlist departuring time            
            min_depTime = get_min_next_depTime(portDAL)
            #remove the flight which will depature firstly
            for arrFlight in portDAL:
                if arrFlight.arrTime + ground_time[arrFlight.arrPort] == min_depTime:
                    portDAL.remove(arrFlight)        
            portDAL.append(flight)
        
    else: 
        #there is empty gate
        if len(portAUS) < gate_no['AUS']:
            portAUS.append(flight)
        else:
            #from cfats which has not stoped flying, find the earlist departuring time            
            min_depTime = get_min_next_depTime(portAUS)
            #remove the flight which will depature firstly
            for arrFlight in portAUS:
                if arrFlight.arrTime + ground_time[arrFlight.arrPort] == min_depTime:
                    portAUS.remove(arrFlight)        
            portAUS.append(flight) 

In [12]:
def get_flight_schdedule():
    
    min_next_depTime = get_min_next_depTime(flights)
    flying = 0 
    
    #there is still a craft which has not stoped flying today
    while flying == 0:
        
        #try to arrange the craft with the minimum possible departure time
        for flight in flights:
                        
            #if the craft is not possible to leave, but its flying status has not been changed
            if flight.arrTime + ground_time[flight.arrPort] > end_time and flight.status == 0:
                flight.status = 1
            
            #rearrange the craft if it's the one with the highest possibility to leave
            elif flight.arrTime + ground_time[flight.arrPort] == min_next_depTime and flight.status == 0:
                                        
                    #get the info for next flight plan            
                    depPort = flight.arrPort
                    depTime = flight.arrTime + ground_time[flight.arrPort]         
                    
                    #if no arrPort available immediately, depTime will be postponed                        
                    arrPort, new_depTime = get_arrPort(depPort, depTime)
            
                    #if there is no available port, depPort == arrPort, new_depTime == end_time
                    if new_depTime != end_time:
                        arrTime = new_depTime + flight_time[(depPort, arrPort)]
                    else:
                        arrTime = end_time + 1            
                    
                    #it's possible to rearrange a flight for the craft
                    if (arrTime <= end_time):                
                        #update aircraft with next flight plan
                        flight.depPort = depPort
                        flight.arrPort = arrPort
                        flight.depTime = new_depTime
                        flight.arrTime = arrTime
                                
                        #update the lists of flights at depPort and arrPort               
                        dep_airport(depPort, flight)
                        arr_airport(arrPort, flight)
            
                        #update flight_schedule    
                        new_flight = copy.deepcopy(flight)
                        flight_schedule.append(new_flight)
                    
                        #update minimum possible depTime
                        min_next_depTime = get_min_next_depTime(flights)                         
                        break
                
                    else:
                        #the craft should stop flying
                        flight.status=1
                        #update minimum possible depTime
                        min_next_depTime = get_min_next_depTime(flights)
                        break
                        
        #find today's flying status
        flying = get_flying_status(flights)
                

In [13]:
#convert time format from minutes to military time
def get_military_time(minutes):
    if minutes < 720:
        hr = minutes // 60 
        min = minutes % 60
        return("{0:02d}{1:02d}".format(hr, min))
    else:
        hr = ((minutes-720) // 60) + 12
        min = (minutes) % 60
        return("{0:02d}{1:02d}".format(hr, min))

In [14]:
#generate flight schedule
get_flight_schdedule()

#sort flight schedule by tailNo and depTime
flight_schedule.sort(key=lambda flight:flight.depTime)
flight_schedule.sort(key=lambda flight:flight.tailNo)

#print flight schedule
for flight in flight_schedule:
    print(flight.tailNo, flight.depPort, flight.arrPort, get_military_time(flight.depTime).zfill(4), get_military_time(flight.arrTime).zfill(4))

T1 AUS HOU 0600 0645
T1 HOU DAL 0720 0825
T1 DAL AUS 0855 0945
T1 AUS DAL 1010 1100
T1 DAL AUS 1130 1220
T1 AUS DAL 1245 1335
T1 DAL AUS 1405 1455
T1 AUS DAL 1520 1610
T1 DAL AUS 1640 1730
T1 AUS HOU 1755 1840
T1 HOU DAL 1915 2020
T1 DAL HOU 2050 2155
T2 DAL HOU 0600 0705
T2 HOU AUS 0740 0825
T2 AUS DAL 0850 0940
T2 DAL AUS 1010 1100
T2 AUS HOU 1125 1210
T2 HOU AUS 1301 1346
T2 AUS HOU 1411 1456
T2 HOU DAL 1531 1636
T2 DAL AUS 1706 1756
T2 AUS HOU 1821 1906
T2 HOU DAL 1941 2046
T3 DAL HOU 0600 0705
T3 HOU DAL 0740 0845
T3 DAL HOU 0915 1020
T3 HOU DAL 1055 1200
T3 DAL AUS 1230 1320
T3 AUS DAL 1345 1435
T3 DAL AUS 1505 1555
T3 AUS DAL 1620 1710
T3 DAL AUS 1740 1830
T3 AUS DAL 1855 1945
T3 DAL AUS 2015 2105
T4 HOU AUS 0600 0645
T4 AUS HOU 0710 0755
T4 HOU AUS 0830 0915
T4 AUS HOU 0940 1025
T4 HOU DAL 1100 1205
T4 DAL HOU 1235 1340
T4 HOU DAL 1415 1520
T4 DAL HOU 1550 1655
T4 HOU DAL 1730 1835
T4 DAL AUS 1905 1955
T4 AUS DAL 2020 2110
T5 HOU DAL 0600 0705
T5 DAL HOU 0735 0840
T5 HOU DAL 09

In [15]:
#the parameters for csv file
file_name = 'flight_schedule.csv'
col_name = 'tail_number,origin,destination,departure_time,arrival_time' 

#export the flight schedule to a csv file
def print_flight_schedule(file_name, col_name, flight_schedule): 
    with open(file_name,'wt') as f:
        print(col_name, file=f) 
        for flight in flight_schedule:
            print(','.join([flight.tailNo, flight.depPort, flight.arrPort, str(get_military_time(flight.depTime)).zfill(4), get_military_time(flight.arrTime).zfill(4)]), file=f)

print_flight_schedule(file_name, col_name, flight_schedule)